# Expansion

Information retrieval systems can be sensitive to phrasing and specific keywords. To mitigate this, one classic retrieval technique is to generate multiple paraphrased versions of a query and return results for all versions of the query. This is called **query expansion**. LLMs are a great tool for generating these alternate versions of a query.

Let's take a look at how we might do query expansion for our Q&A bot over the LangChain YouTube videos, which we started in the [Quickstart](/docs/use_cases/query_analysis/quickstart).

## Setup
#### Install dependencies

In [ ]:
# %pip install -qU langchain langchain-openai

#### Set environment variables

We'll use OpenAI in this example:

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [3]:
[doc.metadata["title"] for doc in docs]

['Self-reflective RAG with LangGraph: Self-RAG and CRAG',
 'WebVoyager',
 'LangGraph: Planning Agents',
 'LangGraph: Multi-Agent Workflows',
 'Streaming Events: Introducing a new `stream_events` method',
 'LangSmith: In-Depth Platform Overview',
 'LangSmith in 10 Minutes',
 'RAG from scratch: Part 8 (Query Translation -- Step Back)',
 'RAG from scratch: Part 9 (Query Translation -- HyDE)',
 'RAG from scratch: Part 7 (Query Translation -- Decomposition - v1)',
 'LangChain Agents with Open Source Models!',
 'Gemini + Google Retrieval Agent from a LangChain Template',
 'OpenGPTs',
 'Building a web RAG chatbot: using LangChain, Exa (prev. Metaphor), LangSmith, and Hosted Langserve',
 'Streaming Events: Introducing a new `stream_events` method',
 'LangGraph: Multi-Agent Workflows',
 'Build and Deploy a RAG app with Pinecone Serverless',
 'Auto-Prompt Builder (with Hosted LangServe)',
 'Build a Full Stack RAG App With TypeScript',
 'Getting Started with Multi-Modal LLMs',
 'SQL Research Assi

In [4]:
docs[0].page_content[:500]

"hi this is Lance from Lang chain I'm going to be talking about using Lang graph to build a diverse and sophisticated rag flows so just to set the stage the basic rag flow you can see here starts with a question retrieval of relevant documents from an index which are passed into the context window of an llm for generation of an answer grounded in the ret documents so that's kind of the basic outline and we can see it's like a very linear path um in practice though you often encounter a few differ"

## Query analysis

To handle these failure modes we can perform **query analysis**. Specifically, we can perform:

* **Query structuring**: If our documents have multiple searchable/filterable attributes, we can infer from any raw user question which specific attributes should be searched/filtered over. For example, when a user input specific something about video publication date, that should become a filter on the `publish_date` attribute of each document.
* **Query decomposition**: If a user input contains multiple distinct questions, we can decompose the input into separate queries.
* **Query expansion**: If an index is sensitive to query phrasing, we can multiple paraphrased versions of the user question to increase our chances of retrieving a relevant result.

To do this we'll define a **query schema** and use a function-calling model to convert a user question into a structured query or queries. The structured nature of the query schema allows us to do query structuring and routing, and the fact that we can extract multiple of these 
allows us to do decomposition and expansion.

### Query schema
In this case we'll have explicit min and max attributes for view count, publication date, and video length so that those can be filtered on. And we'll add separate attributes for searches against the transcript contents versus the video title. We'll also add some sorting attributes that we'll touch on later.

In [67]:
import datetime
from typing import Literal, Optional, Tuple

from langchain_core.pydantic_v1 import BaseModel, Field


class SubQuery(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    sub_query: str = Field(
        ...,
        description="A very specific query against the database.",
    )

### Query generation

To convert user questions to structured queries we'll make use of OpenAI's function-calling API. Since the latest OpenAI models can return multiple function invocations each turn, this approach automatically supports query expansion and decomposition.

In [72]:
from langchain.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a list of database queries optimized to retrieve the most relevant results.

Perform query expansion. If there are multiple common ways of phrasing a user question \
or common synonyms for key words in the question, make sure to return multiple versions \
of the query with the different phrasings.

Perform query decomposition. If the user input contains a multi-part question, make \
sure to return a separate query for each distinct sub-question.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("examples", optional=True),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_with_tools = llm.bind_tools([TutorialSearch])
query_analyzer = (
    {"question": RunnablePassthrough()}
    | prompt
    | llm_with_tools
    | PydanticToolsParser(tools=[TutorialSearch])
)

Let's see what queries our analyzer generates for the questions we searched earlier:

In [73]:
for query in query_analyzer.invoke("rag from scratch"):
    query.pretty_print()
    print()

content_search: RAG from scratch
title_search: RAG
relevance_rank: 1

content_search: Reactive Agile Governance from scratch
title_search: Reactive Agile Governance
relevance_rank: 2

content_search: How to build RAG applications from the beginning
title_search: RAG applications
relevance_rank: 3



In [74]:
for query in query_analyzer.invoke("videos on RAG published in 2023"):
    query.pretty_print()
    print()

content_search: RAG
title_search: RAG
earliest_publish_date: 2023-01-01
latest_publish_date: 2024-01-01
relevance_rank: 1



In [85]:
for query in query_analyzer.invoke(
    "how to use multi-modal models in a chain and turn chain into a rest api"
):
    query.pretty_print()
    print()

content_search: multi-modal models chain
title_search: multi-modal models chain
relevance_rank: 1

content_search: chain into REST API
title_search: chain REST API
relevance_rank: 2



### Improvements: Adding examples to the prompt

To tune our results we can add some examples of inputs questions and gold standard output queries to our prompt. We'll focus on examples that show how to route and expand queries, to either be against titles or content, how to structure them with filters, and how to decompose them:

In [76]:
examples = []

In [77]:
question = "What is Web Voyager? How about Gemini?"
queries = [
    TutorialSearch(
        content_search="what is Web Voyager",
        title_search="Web Voyager",
        relevance_rank=1,
    ),
    TutorialSearch(
        content_search="What is Gemini", title_search="Gemini", relevance_rank=2
    ),
]
examples.append({"input": question, "tool_calls": queries})

In [78]:
question = "Have they released any chat langchain updates since 2024?"
queries = [
    TutorialSearch(
        title_search="chat langchain",
        content_search="chat langchain",
        earliest_publish_date=datetime.date(2024, 1, 1),
        relevance_rank=1,
    ),
]
examples.append({"input": question, "tool_calls": queries})

In [79]:
question = "How to build multi-agent system and stream intermediate steps from it"
queries = [
    TutorialSearch(
        content_search="How to build multi-agent system",
        title_search="multi-agent system",
        relevance_rank=1,
    ),
    TutorialSearch(
        content_search="how to stream intermediate steps from multi-agent system",
        title_search="stream intermediate steps multi-agent system",
        relevance_rank=2,
    ),
    TutorialSearch(
        content_search="how to stream intermediate steps",
        title_search="stream intermediate steps",
        relevance_rank=3,
    ),
]
examples.append({"input": question, "tool_calls": queries})

Now we need to update our prompt template and chain so that the examples are included in each prompt. Since we're working with OpenAI function-calling, we'll need to do a bit of extra structuring to send example inputs and outputs to the model. We'll create a `tool_example_to_messages` helper function to handle this for us:

In [42]:
import uuid
from typing import Dict, List

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)


def tool_example_to_messages(example: Dict) -> List[BaseMessage]:
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    openai_tool_calls = []
    for tool_call in example["tool_calls"]:
        openai_tool_calls.append(
            {
                "id": str(uuid.uuid4()),
                "type": "function",
                "function": {
                    "name": tool_call.__class__.__name__,
                    "arguments": tool_call.json(),
                },
            }
        )
    messages.append(
        AIMessage(content="", additional_kwargs={"tool_calls": openai_tool_calls})
    )
    tool_outputs = example.get("tool_outputs") or [
        "This is an example of a correct usage of this tool. Well done. Make sure to continue using the tool this way."
    ] * len(openai_tool_calls)
    for output, tool_call in zip(tool_outputs, openai_tool_calls):
        messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
    return messages


example_msgs = [msg for ex in examples for msg in tool_example_to_messages(ex)]
query_analyzer_with_examples = (
    {"question": RunnablePassthrough()}
    | prompt.partial(examples=example_msgs)
    | llm_with_tools
    | PydanticToolsParser(tools=[TutorialSearch])
)

In [81]:
for query in query_analyzer_with_examples.invoke("rag from scratch"):
    query.pretty_print()
    print()

content_search: rag from scratch
title_search: rag from scratch
relevance_rank: 1



In [84]:
for query in query_analyzer_with_examples.invoke(
    "how to use multi-modal models in a chain and turn chain into a rest api"
):
    query.pretty_print()
    print()

content_search: how to use multi-modal models in a chain
title_search: multi-modal models chain
relevance_rank: 1

content_search: how to turn chain into a REST API
title_search: chain REST API
relevance_rank: 2



In [83]:
for query in query_analyzer_with_examples.invoke(
    "How to do extraction with agent? How to build agent with anthropic"
):
    query.pretty_print()
    print()

content_search: How to do extraction with agent
title_search: extraction with agent
relevance_rank: 1

content_search: How to build agent with anthropic
title_search: build agent anthropic
relevance_rank: 2

